In [1]:
import pandas as pd
import numpy as np
import os
from typing import Callable # this is to define test_func functions dtype as function itself
import re # for regural expression
import warnings
warnings.filterwarnings('ignore')
import pickle # for pickling my model

# Can import the below dataset from 
/kaggle/input/mobile-recommendation-system-dataset/mobile_recommendation_system_dataset.csv

In [12]:
base_path = os.getcwd()
sub_path = 'data'
dataset_path = os.path.join(base_path, sub_path, 'mobile_recommendation_system_dataset.csv')

In [14]:
df = pd.read_csv(dataset_path,index_col=False)

In [15]:
df.head()

,name,ratings,price,imgURL,corpus
0,"REDMI Note 12 Pro 5G (Onyx Black, 128 GB)",4.2,23999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 SystemAndroid 12Processor T...
1,"OPPO F11 Pro (Aurora Green, 128 GB)",4.5,"₹20,999",https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage256GB S...
2,"REDMI Note 11 (Starburst White, 64 GB)",4.2,13149,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Sp...
3,"OnePlus Nord CE 5G (Blue Void, 256 GB)",4.1,21999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid Q 11Processo...
4,"APPLE iPhone 13 mini (Blue, 128 GB)",4.6,3537,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 SystemiOS 15Processor TypeA15 Bion...


In [16]:
def random_values(col_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [17]:
random_values('corpus')

2005 : Storage256  SystemiOS 16Processor TypeA15 Bionic Chip 12MP 12MP 12MP 12MP 63MP 12MP 12MP 5G notMentioned Display Size17.02 cm (6.7 inch)Resolution2778 x 1284 PixelsResolution TypeSuper Retina XDR DisplayGPU5 CoreDisplay TypeSuper Retina XDR DisplayOther Display FeaturesHDR Display, True Tone, Wide Colour (P3), Haptic Touch, Contrast Ratio: 20,00,000:1, Max Brightness: 800 nits, Peak Brightness: 1,200 nits, Fingerprint Resistant Oleophobic Coating, Support for Display of Multiple Languages and Characters Simultaneously
887 : Storage128 GBRAM6  SystemAndroid 10Processor TypeQualcomm Snapdragon 865Processor Speed2.84 64MP 13MP 5MP 64MP 13MP 5MP 64MP 13MP 5MP 20MP 5G Capacity5000 Display Size16.94 cm (6.67 inch)Resolution2400 x 1080 PixelsResolution TypeFull HD+GPUAdreno 650Display TypeFull HD+ DisplayHD Game SupportYesOther Display FeaturesScreen Protection: Triple Corning Gorilla Glass 5 Covering Front, Back and Camera Lenses, 144 Hz Refresh Rate, Supports Adaptive Sync in 30Hz / 

## Lowercasing the corpus values

In [5]:
df['corpus'] = df['corpus'].str.lower()

In [6]:
random_values('corpus')

738 : storage64 gbram4 gbexpandable storage256gb  systemandroid oreo 8.1processor typequalcomm snapdragon 626processor speed2.2 16mp 5mp 24mp 3g capacity3260 display size16.0 cm (6.3 inch)resolution2280 x 1080 pixelsresolution typefull hd+gpuadreno 506display typeips capacitive touch screenother display featuresfullview display 2.0, 1.75 mm ultra-slim bezel, 90% screen-to-body ratio, face scan to unlock, face scan for app-lock, smart motion, corning gorilla 2.5d glass , eye protection mode, smart split 3.0, 19:9 display
215 : storage128 gbram6  systemandroid q 12processor speed2.2 50mp 4g capacity6000 display size16.76 cm (6.6 inch)resolution720x2408 pixel
1230 : storage128 gbram6 gbexpandable storage1tb  systemandroid 11processor typequalcomm snapdragon 662processor speed2 48mp 2mp 2mp 48mp 2mp 2mp 48mp 2mp 2mp 48mp 16mp 16mp 4g capacity5000 display size16.71 cm (6.58 inch)resolution2408 x 1080 pixelsresolution typefull hd+gpuadreno 610display typefull hd+ lcd full-incell ips displayo

In [18]:
df.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2546 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2546 non-null   object 
 1   ratings  2546 non-null   float64
 2   price    2546 non-null   object 
 3   imgURL   2546 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 99.6+ KB


In [20]:
df.dropna(inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2534 entries, 0 to 2545
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     2534 non-null   object 
 1   ratings  2534 non-null   float64
 2   price    2534 non-null   object 
 3   imgURL   2534 non-null   object 
 4   corpus   2534 non-null   object 
dtypes: float64(1), object(4)
memory usage: 118.8+ KB


In [22]:
max_len = 0
for i in range(0, len(df['corpus'])):
    try:
        if len(df['corpus'][i]) > max_len:
            max_len = len(df['corpus'][i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max len in corpus is: 1196


In [23]:
df.sample(10)

,name,ratings,price,imgURL,corpus
1584,"Redmi K20 Pro (Carbon Black, 256 GB)",4.5,31999,https://rukminim2.flixcart.com/image/312/312/j...,Storage256 GBRAM8 SystemAndroid Pie 9.0Proces...
1960,"SAMSUNG Galaxy F12 (Celestial Black, 128 GB)",3.6,11499,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM4 GBExpandable Storage512GB S...
2536,"OnePlus 7 Pro (Nebula Blue, 256 GB)",4.2,29999,https://rukminim2.flixcart.com/image/312/312/k...,Storage256 GBRAM12 SystemAndroid Pie 9 Pie Ox...
88,"realme Narzo 50i Prime (Mint Green, 64 GB)",4.2,9989,https://rukminim2.flixcart.com/image/312/312/x...,Storage64 GBRAM4 SystemAndroid 11Processor Ty...
1964,"vivo V25 Pro 5G (Pure Black, 128 GB)",4.3,29537,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM8 SystemAndroid 12Processor T...
977,"APPLE iPhone 6 (Grey, 128 GB)",4.6,5964,https://rukminim2.flixcart.com/image/312/312/m...,Storage128 SystemiOS 8Processor TypeApple A8 6...
1685,"Xiaomi 12 Pro 5G (Couture Blue, 256 GB)",4.2,20537,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12 SystemAndroid 12Processor ...
349,"SAMSUNG Galaxy A22 5G (Gray, 128 GB)",4.1,23999,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM6 GBExpandable Storage1TB Sys...
1444,"Tecno Spark Air 6 (Cloud White, 32 GB)",4.1,9499,https://rukminim2.flixcart.com/image/312/312/k...,Storage32 GBRAM2 SystemAndroid Q 10.1Processo...
1997,"APPLE iPhone 5C (Green, 8 GB)",3.9,28036,https://rukminim2.flixcart.com/image/312/312/k...,Storage8 SystemiOS 7Processor TypeA6 Chip 8MP...


In [24]:
random_values('imgURL')

1421 : https://rukminim2.flixcart.com/image/312/312/kavefm80/mobile/w/u/z/samsung-galaxy-m11-sm-m115fmbeins-original-imafscgycugcewty.jpeg?q=70
1977 : https://rukminim2.flixcart.com/image/312/312/kg8avm80/mobile/p/f/w/apple-iphone-12-dummyapplefsn-original-imafwg8dubf3nytw.jpeg?q=70
1753 : https://rukminim2.flixcart.com/image/312/312/kdxfc7k0/mobile/j/6/2/realme-c15-rmx2180-original-imafupu9wutkbf2k.jpeg?q=70
1584 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/q/n/m/-original-image7d9zgguhgcy.jpeg?q=70
382 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/j/7/q/galaxy-a03-core-sm-a032fck6ins-samsung-original-imagmb5kvceyb2yc.jpeg?q=70
141 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/9/o/l/-original-imagdh2zh7hbwyxh.jpeg?q=70


# Model

## vectorization(to convert the words to vectors)

In [12]:

from sklearn.feature_extraction.text import CountVectorizer 

In [13]:
cont_vect = CountVectorizer(max_features=1000)

In [14]:
arrays = cont_vect.fit_transform(df['corpus'])

In [15]:
vectors = arrays.toarray()

In [16]:
cont_vect.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '07', '07bother', '08', '09',
       '0mp', '0primary', '0processor', '10', '100', '1000', '100000',
       '1000000', '100mp', '103', '104', '106', '1080', '1080processor',
       '1080resolution', '108mp', '10mp', '10processor', '11', '1100',
       '1125', '1170', '1179', '11primary', '11processor', '12', '120',
       '1200', '1200processor', '120hz', '1242', '1280processor', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300',
       '1300processor', '1330', '1334', '135', '13mp', '13processor',
       '14', '1400', '144', '1440', '144hz', '1480', '14processor', '15',
       '1500', '1544', '15processor', '16', '160', '1600', '16000',
       '1600resolution', '1612', '16mp', '16processor', '18', '180',
       '1800', '180hz', '19', '1920', '1920hz', '1processor', '20', '200',
       '2000000', '200mp', '20mp', '21', '22', '23', '2340', '24', '240',
       '2400', '2400resolution', '2408', '240hz', '2460', '24mp', '25m

## Calculating cosine similarity for recommendation

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
similarity = cosine_similarity(vectors)
similarity.shape # this is because I have calculated the similarity among all the rows to all the rows(confusing?? but try to understand)

(2534, 2534)

In [19]:
similarity[3]

array([0.3345748 , 0.17485208, 0.53136893, ..., 0.57564968, 0.27586342,
       0.37283655])

In [20]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(68, 0.9999999999999997),
 (303, 0.9999999999999997),
 (616, 0.9850746268656713),
 (1461, 0.9850746268656713),
 (457, 0.970149253731343),
 (1027, 0.970149253731343),
 (1735, 0.970149253731343),
 (569, 0.9468131937613656),
 (1123, 0.9468131937613656),
 (1540, 0.9468131937613656)]

In [21]:
def recommend(mobile):
    mobile_index = df[df['name']==mobile].index[0]
    similarity_array = similarity[mobile_index]
    similar_10_mobiles = sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in similar_10_mobiles:
        print(df['name'].iloc[i[0]])



In [22]:
recommend('APPLE iPhone 13 mini (Blue, 128 GB)')

APPLE iPhone 13 mini ((PRODUCT)RED, 128 GB)
APPLE iPhone 13 mini (Starlight, 128 GB)
APPLE iPhone 13 mini (Green, 128 GB)
APPLE iPhone 13 mini (Midnight, 128 GB)
APPLE iPhone 13 mini (Green, 256 GB)
APPLE iPhone 13 mini (Midnight, 512 GB)
APPLE iPhone 13 mini (Pink, 512 GB)
APPLE iPhone 13 mini (Starlight, 256 GB)
APPLE iPhone 13 mini (Midnight, 256 GB)
APPLE iPhone 13 mini (Pink, 256 GB)


## making Pickle file of df and similarities to use in web

In [23]:
os.makedirs(r'src\model',exist_ok=True)

In [24]:
pickle.dump(obj=df,file=open(file=r'src\model\dataframe.pkl',mode='wb'))

In [25]:
pickle.dump(obj=similarity,file=open(file=r'src\model\similarity.pkl',mode='wb'))